<a href="https://colab.research.google.com/github/Jacobgokul/ML-Playground/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## What is RNN?
- RNN stands for Recurrent Neural Network.

- It’s a type of neural network designed to handle sequential data, where the order matters.

- Unlike standard neural networks, RNNs remember previous inputs using hidden states, making them perfect for sequences like text, speech, or time series.

- Key Idea:
    RNNs process data step by step, using the previous step’s output (hidden state) as context for the next step.

## How RNN Works

- Input Sequence: The input is processed one step at a time.

- Hidden State: At each step, the model updates a hidden state that stores information about past inputs.

- Recurrent Connection: The output from the previous step is passed as an input to the next step.

- Output: After processing all steps, the network provides an output (classification, prediction, etc.).

## Why Use RNN?
- Perfect for sequential/temporal data: order matters.

- Captures dependencies across time or sequence.

- Can generate sequences (text, music) or predict future steps.

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

In [2]:
# Example text corpus
corpus = [
    "hello how are you",
    "hello how is your day",
    "hello how are your friends",
    "hello what are you doing"
]

In [3]:
# Tokenize words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1
print("Total unique words:", total_words)

Total unique words: 11


In [5]:
# Create input sequences
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [6]:
# Pad sequences to same length
max_seq_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_seq_len, padding='pre'))

# Split inputs and labels
X = input_sequences[:,:-1]
y = input_sequences[:,-1]

print("Example X[0]:", X[0], "-> y[0]:", y[0])

Example X[0]: [0 0 0 1] -> y[0]: 2


Explanation:

- Tokenizer converts words → integers.

- Input sequences: previous words predict the next word.

- Padding ensures all sequences have the same length.

In [7]:
# Build RNN Model
model = Sequential()

# Embedding layer: converts word indices into dense vectors
model.add(Embedding(input_dim=total_words, output_dim=10, input_length=max_seq_len-1))

# RNN layer: learns sequence patterns
model.add(SimpleRNN(50, activation='relu'))

# Output layer: predicts next word
model.add(Dense(total_words, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


d:\A Code\ML-Playground\.venv\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [8]:
history = model.fit(X, y, epochs=200, verbose=0)
print("Training complete!")


Training complete!


Loss: sparse categorical cross-entropy because labels are integers.

Optimizer: Adam for faster convergence.

Epochs: 200 (small dataset).

In [15]:
def predict_next_word(model, tokenizer, text_seq, max_seq_len):
    token_list = tokenizer.texts_to_sequences([text_seq])[0]
    token_list = pad_sequences([token_list], maxlen=max_seq_len-1, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    predicted_word_index = np.argmax(predicted)
    
    for word, index in tokenizer.word_index.items():
        if index == predicted_word_index:
            return word

# Test the model
seed_text = "hello how is"
next_word = predict_next_word(model, tokenizer, seed_text, max_seq_len)
print(f"Input: '{seed_text}' → Predicted next word: '{next_word}'")


Input: 'hello how is' → Predicted next word: 'your'


Explanation:

- Converts input text → integer sequence.

- Pads sequence to match training input length.

- Predicts next word based on RNN memory of previous words.

#### Key Takeaways:

- RNNs are great for sequences like text, audio, and time series.

- Hidden states carry memory from previous timesteps.

- Embedding layer converts words into dense vector representation.

- SimpleRNN processes the sequence step by step.

- Dense + softmax outputs probability of the next word.

In [ ]:
"""
Input Sequence:        "hello"     "how"      "are"
                        x1          x2         x3
                         |           |          |
                     Embedding    Embedding  Embedding
                         |           |          |
                         e1          e2         e3
                         |           |          |
Hidden States:          h0 --------> h1 --------> h2 --------> h3
                         |           |          |
                         ----------------------- 
                                     |
                                output layer + Softmax
                                     |
                             Predicted next word
"""

'\nInput Sequence:        "hello"     "how"      "are"\n                        x1          x2         x3\n                         |           |          |\n                     Embedding    Embedding  Embedding\n                         |           |          |\n                         e1          e2         e3\n                         |           |          |\nHidden States:          h0 --------> h1 --------> h2 --------> h3\n                         |           |          |\n                         ----------------------- \n                                     |\n                                Dense + Softmax\n                                     |\n                             Predicted next word\n'